In [4]:
import numpy as np

%run functions/preprocess.py

np.random.seed(42)
pd.set_option('display.float_format', lambda x: '%.3f' % x)


### Time preprocessing

In [5]:
% % time
path = open('path.txt', 'r').read().strip()
train = pd.read_csv(f'{path}BPI Challenge 2017-training.csv')
test = pd.read_csv(f'{path}BPI Challenge 2017-test.csv')
data = pd.concat([train, test])

# strip whitespace of column names and add extra time columns
data.columns = data.columns.str.strip()

# remove milliseconds
remove_ms(data)

# Convert to timestamp and decompose features
data['event time:timestamp'] = f_memoize_dt(data['event time:timestamp'])
data['time of day'] = data['event time:timestamp'].dt.time
data["weekday"] = data["event time:timestamp"].dt.day_name()

# sorting values
data.sort_values(by=['event time:timestamp'], inplace=True)
data.reset_index(inplace=True, drop=True)

# add unix time column
UnixTime(data)
data["Unix"] = data["Unix"].astype(int)

# drop useless columns
dropper(data, lbls=["eventID", "event EventID"])


# check if dataframe sorted
# for i in range(0,1202267, 10000):
#     display(df_data['event time:timestamp'].iloc[[i]])

######## CHECK IF CORRECT #########
# data["timestamp"] = data["event time:timestamp"].copy()
# data["event time:timestamp"] = pd.to_datetime(data["event time:timestamp"], dayfirst=True)
# data['time of day'] = data["timestamp"].str.split(expand=True)[1]


CPU times: user 13 s, sys: 1.49 s, total: 14.5 s
Wall time: 14.5 s


### Event/time adder

In [6]:
# event shifting
data['next_event'] = data.groupby('case concept:name')[
    'event concept:name'].shift(-1)
data["prev_event"] = data.groupby('case concept:name')[
    'event concept:name'].shift(1)

# time shifting
data["nextUnix"] = data.groupby('case concept:name')['Unix'].shift(-1)
data["prevUnix"] = data.groupby('case concept:name')['Unix'].shift(1)
data["nextTime"] = data.groupby('case concept:name')[
    'event time:timestamp'].shift(-1)
data["prevTime"] = data.groupby('case concept:name')[
    'event time:timestamp'].shift(1)

# Adding time features
data['day'] = data['event time:timestamp'].dt.day
data['month'] = data['event time:timestamp'].dt.month
data['hour'] = data['event time:timestamp'].dt.hour
data['day_of_week'] = data['event time:timestamp'].dt.weekday

# Cyclical encoding
data["hour"] = 2 * np.pi * data["hour"] / data["hour"].max()
data["hour_cos"] = np.cos(data["hour"])
data["hour_sin"] = np.sin(data["hour"])
data["day_of_week"] = 2 * np.pi * \
    data["day_of_week"] / data["day_of_week"].max()
data["day_of_week_cos"] = np.cos(data["day_of_week"])
data["day_of_week_sin"] = np.sin(data["day_of_week"])


### Encoding

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(data[['case RequestedAmount']])
data['case RequestedAmount'] = scaler.transform(data[['case RequestedAmount']])


In [8]:
data.to_pickle(f"processed2017.pkl")
